In [4]:
scan_id = 'PS14_001'

import os, sys  # system functions
import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.interfaces.utility as util  # utility
import nipype.interfaces.ants as ants
from nipype.interfaces.c3 import C3dAffineTool
from nipype.interfaces.ants.segmentation import BrainExtraction

import multiprocessing, time
from multiprocessing import Pool
nprocs=multiprocessing.cpu_count()

start_time = time.time()


In [5]:
fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

In [7]:

main_dir = "/media/data/EPI_REG/"
project_dir = main_dir+"/Analysis/"
work_dir = main_dir+"/scratch/"

if not os.path.exists(work_dir):
    os.makedirs(work_dir)

template_brain = os.path.join(project_dir,'MNI','nihpd_asym_04.5-08.5_t1w.nii')
moving_image = project_dir+"/"+scan_id+"/PS14_001_T1w_n4.nii"



In [8]:
template_brain

'/media/data/EPI_REG//Analysis/MNI/nihpd_asym_04.5-08.5_t1w.nii'

In [9]:
moving_image

'/media/data/EPI_REG//Analysis//PS14_001/PS14_001_T1w_n4.nii'

In [12]:

"""
Create workflow
"""

wf = pe.Workflow(name='wf')
wf.base_dir = os.path.join(work_dir, "reg_wdir", scan_id)
wf.config = {"execution": {"crashdump_dir": os.path.join(work_dir, 'reg_crashdumps', scan_id)}}

"""
Register T1 to MNI
"""
reg = pe.Node(ants.Registration(), name='antsRegister')
reg.inputs.output_transform_prefix = "anat2mni_"
reg.inputs.transforms = ['Rigid', 'Affine', 'SyN']
reg.inputs.transform_parameters = [(0.1,), (0.1,), (0.2, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[10000,111110,11110]] * 2 + [[100, 100, 50]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = True
reg.inputs.initial_moving_transform_com = True
reg.inputs.metric = ['Mattes'] * 2 + [['Mattes', 'CC']]
reg.inputs.metric_weight = [1] * 2 + [[0.5, 0.5]]
reg.inputs.radius_or_number_of_bins = [32] * 2 + [[32, 4]]
reg.inputs.sampling_strategy = ['Regular'] * 2 + [[None, None]]
reg.inputs.sampling_percentage = [0.3] * 2 + [[None, None]]
reg.inputs.convergence_threshold = [1.e-7] * 2 + [-0.01]
reg.inputs.convergence_window_size = [20] * 2 + [5]
reg.inputs.smoothing_sigmas = [[4, 2, 1]] * 2 + [[1, 0.5, 0]]
reg.inputs.sigma_units = ['vox'] * 3
reg.inputs.shrink_factors = [[3, 2, 1]]*2 + [[4, 2, 1]]
reg.inputs.use_estimate_learning_rate_once = [True] * 3
reg.inputs.use_histogram_matching = [False] * 2 + [True]
reg.inputs.winsorize_lower_quantile = 0.005
reg.inputs.winsorize_upper_quantile = 0.995
reg.inputs.args = '--float'
reg.inputs.output_warped_image = 'anat2mni_warped_image.nii.gz'
reg.inputs.output_inverse_warped_image = 'mni2anat_warped_image.nii.gz'
reg.inputs.num_threads = 2

reg.inputs.fixed_image=template_brain
reg.inputs.moving_image=moving_image

In [27]:

"""
Transform MNI mask, which is aligned to T1, to MNI
"""

warp_mask = pe.MapNode(ants.ApplyTransforms(), iterfield=['input_image','transforms'], name='warp_mask')
warp_mask.inputs.input_image_type = 0
warp_mask.inputs.interpolation = 'Linear'
warp_mask.inputs.invert_transform_flags = [False]
# warp_mask.inputs.terminal_output = 'file'
warp_mask.inputs.input_image = os.path.join(project_dir,'MNI','nihpd_asym_04.5-08.5_mask.nii')

wf.connect(reg, 'inverse_composite_transform', warp_mask, 'transforms') # using transform matrix from t1 to mni
warp_mask.inputs.reference_image = moving_image


In [29]:

"""
Save data
"""

datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = os.path.join(project_dir, "reg")

datasink.inputs.container = scan_id

wf.connect(reg, 'warped_image', datasink, 'anat.anat2mni')
wf.connect(reg, 'inverse_warped_image', datasink, 'anat.mni2anat')
wf.connect(reg, 'composite_transform', datasink, 'anat.anat2mni_mat')
wf.connect(reg, 'inverse_composite_transform', datasink, 'anat.mni2anat_mat')
wf.connect(warp_mask, 'output_image', datasink, 'anat.mask2t1')


In [30]:

"""
Run
"""
outgraph = wf.run(plugin='MultiProc')

print("--- %s seconds ---" % (time.time() - start_time))


180306-17:56:39,94 workflow INFO:
	 Workflow wf settings: ['check', 'execution', 'logging', 'monitoring']
180306-17:56:39,108 workflow INFO:
	 Running in parallel.
180306-17:56:39,121 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.21/28.21, Free processors: 6/6.
180306-17:56:39,230 workflow INFO:
	 [Node] Setting-up "wf.antsRegister" in "/media/data/EPI_REG/scratch/reg_wdir/PS14_001/wf/antsRegister".
180306-17:56:39,247 workflow INFO:
	 [Node] Running "antsRegister" ("nipype.interfaces.ants.registration.Registration"), a CommandLine Interface with command:
antsRegistration --float --collapse-output-transforms 1 --dimensionality 3 --initial-moving-transform [ /media/data/EPI_REG//Analysis/MNI/nihpd_asym_04.5-08.5_t1w.nii, /media/data/EPI_REG//Analysis//PS14_001/PS14_001_T1w_n4.nii, 1 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ anat2mni_, anat2mni_warped_image.nii.gz, mni2anat_warped_image.nii.gz ] --transform Rigid[ 0.1 ]

In [25]:
warp_mask.inputs.terminal_output

'file'

In [31]:
pwd

'/home/hongfu'